In [2]:
import pandas as pd

In [3]:
import plotly.express as px
import plotly.graph_objects as go

In [4]:
cindy_weather = pd.read_csv('./dataset/weather.csv')

In [5]:
cindy_weather.shape

(5495, 24)

In [19]:
cindy_weather['zip_code'].unique()

array([94107, 94063, 94301, 94041, 95113], dtype=int64)

In [38]:
zip2latlong = {94107:(37.7680296,-122.4096411), 
               94063:(37.5018464,-122.240711), 
               94301:(37.4417477,-122.1685752), 
               94041:(37.4130314,-122.0977141), 
               95113:(37.3331982,-121.9014242)}

In [20]:
station1 = pd.read_csv('./dataset/train_station.csv')
station2 = pd.read_csv('./dataset/test_station.csv')

In [22]:
all_station = pd.concat([station1,station2],sort=False)

In [31]:
all_station = all_station.groupby('name').agg('first')

In [36]:
lat_long = all_station[['lat','long']].to_numpy()

In [42]:
res = []
for l in lat_long:
    mindistzip = -1
    mindist = 100000
    for z in zip2latlong:
        dic = zip2latlong[z]
        d = (dic[0]-l[0])**2 + (dic[1]-l[1])**2
        if d < mindist:
            mindist = d
            mindistzip = z
    res.append(mindistzip)


all_station['city'] = res
station_city_dict = dict((k,v) for k,v in zip(all_station.index, res))

In [53]:
station_city_dict

{'2nd at Folsom': 94107,
 '2nd at South Park': 94107,
 '2nd at Townsend': 94107,
 '5th S. at E. San Salvador St': 95113,
 '5th St at Folsom St': 94107,
 '5th at Howard': 94107,
 'Adobe on Almaden': 95113,
 'Arena Green / SAP Center': 95113,
 'Beale at Market': 94107,
 'Broadway St at Battery St': 94107,
 'California Ave Caltrain Station': 94301,
 'Castro Street and El Camino Real': 94041,
 'Charleston Park/ North Bayshore Area': 94041,
 'Civic Center BART (7th at Market)': 94107,
 'Clay at Battery': 94107,
 'Commercial at Montgomery': 94107,
 'Cowper at University': 94301,
 'Cyril Magnin St at Ellis St': 94107,
 'Davis at Jackson': 94107,
 'Embarcadero at Bryant': 94107,
 'Embarcadero at Folsom': 94107,
 'Embarcadero at Sansome': 94107,
 'Embarcadero at Vallejo': 94107,
 'Evelyn Park and Ride': 94041,
 'Franklin at Maple': 94063,
 'Golden Gate at Polk': 94107,
 'Grant Avenue at Columbus Avenue': 94107,
 'Harry Bridges Plaza (Ferry Building)': 94107,
 'Howard at 2nd': 94107,
 'Japantown

In [44]:
all_trips = pd.read_csv('./dataset/all_trips.csv')

In [63]:
trip_stations = all_trips['start_station_name'].unique()

In [54]:
out_of_list_mapping = {}

for i in trip_stations:
    if i not in station_city_dict:
        out_of_list_mapping[i]=""

In [59]:
all_trips[all_trips['start_station_name'].isin(out_of_list_mapping.keys())].groupby('start_station_name').size()

start_station_name
5th S at E. San Salvador St        19
5th St at E. San Salvador St        1
Broadway at Main                   67
Kaiser Hospital                   147
Mezes                             119
Post at Kearny                  11308
S. Market St at Park Ave           84
San Jose Government Center         23
Sequoia Hospital                   15
Washington at Kearny             7136
dtype: int64

In [55]:
out_of_list_mapping

{'5th S at E. San Salvador St': '',
 '5th St at E. San Salvador St': '',
 'Broadway at Main': '',
 'Kaiser Hospital': '',
 'Mezes': '',
 'Post at Kearny': '',
 'S. Market St at Park Ave': '',
 'San Jose Government Center': '',
 'Sequoia Hospital': '',
 'Washington at Kearny': ''}

In [60]:
out_of_list_mapping['5th S at E. San Salvador St'] = '5th S. at E. San Salvador St'
out_of_list_mapping['5th St at E. San Salvador St'] = '5th S. at E. San Salvador St'
out_of_list_mapping['Broadway at Main'] = 'Broadway St at Battery St'
out_of_list_mapping['Post at Kearny'] = 'Washington at Kearney'
out_of_list_mapping['Washington at Kearny'] = 'Washington at Kearney'
# Can't find corresponding station, but its in SF anyway
out_of_list_mapping['Kaiser Hospital'] = 'San Francisco City Hall'
out_of_list_mapping['Mezes'] = 'Mezes Park'
out_of_list_mapping['S. Market St at Park Ave'] = 'S. Market st at Park Ave'
# Government Center is in fucking phillipin
out_of_list_mapping['San Jose Government Center'] = 'San Jose City Hall'
# Its in redwood city
out_of_list_mapping['Sequoia Hospital'] = 'Stanford in Redwood City'

In [61]:
out_of_list_mapping

{'5th S at E. San Salvador St': '5th S. at E. San Salvador St',
 '5th St at E. San Salvador St': '5th S. at E. San Salvador St',
 'Broadway at Main': 'Broadway St at Battery St',
 'Kaiser Hospital': 'San Francisco City Hall',
 'Mezes': 'Mezes Park',
 'Post at Kearny': 'Washington at Kearney',
 'S. Market St at Park Ave': 'S. Market st at Park Ave',
 'San Jose Government Center': 'San Jose City Hall',
 'Sequoia Hospital': 'Stanford in Redwood City',
 'Washington at Kearny': 'Washington at Kearney'}

In [62]:
all_trips.replace({'start_station_name': out_of_list_mapping}, inplace=True)

In [65]:
all_trips['start_station_name'].unique().shape

(76,)

In [68]:
cty_zip = []
for i in all_trips['start_station_name']:
    cty_zip.append(station_city_dict[i])
len(cty_zip)

983648

In [69]:
all_trips['city_zip'] = cty_zip

In [70]:
all_trips.to_csv('./dataset/cityzip_trips.csv')